## Import libraries

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import glob
import os

### Import Data - Random Sampling (n=5000 per year between Y2015 - Y2017)

In [5]:
data_PartB = pd.read_csv('./dataset2/PartB/Medicare_Provider_Util_Payment_PUF_CY2015.txt', sep='\t')
idxs_PartB = np.random.choice(data_PartB.shape[0], size=50000, replace=False)
data_PartB = data_PartB.iloc[idxs_PartB]
data_PartB.to_csv(r'./dataset2/PartB/data_PartB_2015v2.csv')

C:\Users\Serene\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
#del data_PartB

### Combined all CSV for LEIE Dataset

In [9]:
#os.chdir("./dataset2/LEIE_Dataset")
#extension = 'csv'
#all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
##combine all files in the list
#combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
##export to csv
#combined_csv.to_csv( "LEIE_2018.csv", index=False, encoding='utf-8-sig')

### Combined all CSV for respective Dataset

In [11]:
#os.chdir("./dataset2/PartB")
#extension = 'csv'
#all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
##combine all files in the list
#combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
##export to csv
#combined_csv.to_csv( "PartB.csv", index=False, encoding='utf-8-sig')

### Read Dataset

In [13]:
LEIE = pd.read_csv('./dataset2/LEIE_2018.csv') #contains physicians who are unsuited to practice medicine 
PartB = pd.read_csv('./dataset2/PartB.csv') #provides information on services and procedures provided to Medicare beneficiaries by physicians and other healthcare professionals  

C:\Users\Serene\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Serene\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
## Lets Check Shape of datasets 
print('Shape of LEIE :',LEIE.shape)
print('Shape of PartB :', PartB.shape)

Shape of LEIE : (2509, 18)
Shape of PartB : (150000, 27)
Shape of PartD : (150000, 22)
Shape of DMEPOS : (150000, 29)


In [15]:
#change column names to lower case
LEIE.columns = map(str.lower, LEIE.columns)
PartB.columns = map(str.lower, PartB.columns)

In [16]:
#exclusion due to fraud
LEIE = LEIE[(LEIE.excltype == "1128a1") |(LEIE.excltype == "1128a2") | (LEIE.excltype == "1128a3") | (LEIE.excltype == "1128b4")| (LEIE.excltype == "1128b1") | (LEIE.excltype == "1128b7")]

In [17]:
LEIE.columns

Index(['lastname', 'firstname', 'midname', 'busname', 'general', 'specialty',
       'upin', 'npi', 'dob', 'address', 'city', 'state', 'zip', 'excltype',
       'excldate', 'reindate', 'waiverdate', 'wvrstate'],
      dtype='object')

In [18]:
#drop invalid npi
LEIE = LEIE[LEIE.npi != 0]

In [19]:
LEIE["fraud"] = 1

In [20]:
#drop provider-related information that are not likely to contribute to fraud detection. 
LEIE = LEIE.drop(['lastname', 'firstname', 'midname', 'busname', 'general', 'specialty',
       'upin', 'dob', 'address', 'city', 'state', 'zip',
       'excldate', 'reindate', 'waiverdate', 'wvrstate','excltype'],axis=1)

###  MERGE PART B & LEIE

In [21]:
Combined_B = pd.merge(PartB, LEIE, on='npi', how='left')

In [22]:
#medicare_participation_indicator - drop records that did not participate in medicare
Combined_B = Combined_B[Combined_B.medicare_participation_indicator != "N"]

In [23]:
#drop provider-related information and redundant variables that are not likely to contribute to fraud detection. 
Combined_B = Combined_B.drop(['unnamed: 0', "medicare_participation_indicator",
       'nppes_credentials', 'nppes_entity_code', "hcpcs_description", 'hcpcs_description', 'hcpcs_code',
        'nppes_provider_country', 'nppes_provider_first_name', 'nppes_provider_last_org_name',
       'nppes_provider_mi', 'nppes_provider_street1','nppes_provider_city','nppes_provider_state',
       'nppes_provider_street2', 'nppes_provider_zip', 'place_of_service'],axis=1)

In [24]:
Combined_B.columns

Index(['average_medicare_allowed_amt', 'average_medicare_payment_amt',
       'average_medicare_standard_amt', 'average_submitted_chrg_amt',
       'bene_day_srvc_cnt', 'bene_unique_cnt', 'hcpcs_drug_indicator',
       'line_srvc_cnt', 'npi', 'nppes_provider_gender', 'provider_type',
       'fraud'],
      dtype='object')

In [25]:
Combined_B.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149946 entries, 0 to 149999
Data columns (total 12 columns):
average_medicare_allowed_amt     149946 non-null float64
average_medicare_payment_amt     149946 non-null float64
average_medicare_standard_amt    149946 non-null float64
average_submitted_chrg_amt       149946 non-null float64
bene_day_srvc_cnt                149946 non-null int64
bene_unique_cnt                  149946 non-null int64
hcpcs_drug_indicator             149946 non-null object
line_srvc_cnt                    149946 non-null float64
npi                              149946 non-null int64
nppes_provider_gender            143361 non-null object
provider_type                    149946 non-null object
fraud                            14 non-null float64
dtypes: float64(6), int64(3), object(3)
memory usage: 14.9+ MB


In [26]:
#filling null values
Combined_B["nppes_provider_gender"].fillna("NA", inplace=True)
Combined_B["fraud"].fillna(0, inplace=True)

In [27]:
Combined_B.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149946 entries, 0 to 149999
Data columns (total 12 columns):
average_medicare_allowed_amt     149946 non-null float64
average_medicare_payment_amt     149946 non-null float64
average_medicare_standard_amt    149946 non-null float64
average_submitted_chrg_amt       149946 non-null float64
bene_day_srvc_cnt                149946 non-null int64
bene_unique_cnt                  149946 non-null int64
hcpcs_drug_indicator             149946 non-null object
line_srvc_cnt                    149946 non-null float64
npi                              149946 non-null int64
nppes_provider_gender            149946 non-null object
provider_type                    149946 non-null object
fraud                            149946 non-null float64
dtypes: float64(6), int64(3), object(3)
memory usage: 14.9+ MB


In [28]:
Combined_B.to_csv("Combined_B_2017v3", encoding='utf-8-sig')